<a href="https://colab.research.google.com/github/sumankrsh/Breast-Cancer-Detection-Model/blob/main/Breast_Cancer_Detection_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Step 1: Installation
# Step 2: Setting up the Configs
# Step 3: Initializing the Model and Training
# Step 4: Evaluating the Model on unseen data
# Step 5: Saving the Model

In [4]:
import warnings 
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

In [5]:
# reading & Loading the data
from google.colab import files
uploaded = files.upload()


Saving data.csv to data.csv


In [6]:
# reading data
data=pd.read_csv('data.csv')
data['diagnosis'] = data['diagnosis'].astype('category').cat.codes# encoding labels
data.drop(['id','Unnamed: 32'],axis=1,inplace=True)

In [7]:
def load_classification_data(df,target_col,test_size):
    torch_data = np.array(df.drop(target_col,axis=1))
    torch_labels =np.array(df[target_col])
    data = np.hstack([torch_data, torch_labels.reshape(-1,1)])
    gen_names = [f"feature_{i}" for i in range(data.shape[-1])]
    col_names = gen_names
    col_names[-1] = "target"
    data = pd.DataFrame(data, columns=col_names)
    cat_col_names = [x for x in gen_names[:-1] if len(data[x].unique())<10]
    num_col_names = [x for x in gen_names[:-1] if x not in [target_col]+cat_col_names]
    test_idx = data.sample(int(test_size * len(data)), random_state=42).index
    test = data[data.index.isin(test_idx)]
    train = data[~data.index.isin(test_idx)]
    return (train, test, ["target"],cat_col_names,num_col_names)

In [8]:
train, test, target_col, cat_col_names, num_col_names= load_classification_data(data,'diagnosis',0.2)

In [9]:
! pip install pytorch_tabular[all]

     |████████████████████████████████| 1.9 MB 8.5 MB/s 
     |████████████████████████████████| 80 kB 8.9 MB/s 
     |████████████████████████████████| 6.8 MB 57.3 MB/s 
     |████████████████████████████████| 809 kB 56.9 MB/s 
     |████████████████████████████████| 74 kB 3.1 MB/s 
     |████████████████████████████████| 272 kB 65.4 MB/s 
     |████████████████████████████████| 636 kB 39.6 MB/s 
     |████████████████████████████████| 1.8 MB 45.6 MB/s 
     |████████████████████████████████| 13.2 MB 35.1 MB/s 
     |████████████████████████████████| 118 kB 62.0 MB/s 
     |████████████████████████████████| 829 kB 64.2 MB/s 
     |████████████████████████████████| 170 kB 46.2 MB/s 
     |████████████████████████████████| 75 kB 3.8 MB/s 
     |████████████████████████████████| 133 kB 49.1 MB/s 
     |████████████████████████████████| 97 kB 6.2 MB/s 
     |████████████████████████████████| 1.3 MB 44.7 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |██████████████████

In [10]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, NodeConfig, TabNetModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from pytorch_tabular.categorical_encoders import CategoricalEmbeddingTransformer

In [11]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=32,
    max_epochs=100,
    gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="4096-4096-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3,
    metrics=["accuracy"]
)
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

In [12]:
tabular_model.fit(train=train, test=test)

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 0     
1 | normalizing_batch_norm | BatchNorm1d         | 60    
2 | backbone               | FeedForwardBackbone | 19.0 M
3 | output_layer           | Linear              | 1.0 K 
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
19.0 M    Trainable params
0         Non-trainable params
19.0 M    Total params
76.028    Total estimated model params size (MB)
Global seed set to 42


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
-------------------------

LR finder stopped early after 82 steps due to diverging loss.
Restored states from the checkpoint file at /content/lr_find_temp_model.ckpt
Learning rate set to 2.7542287033381663e-05
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 0     
1 | normalizing_batch_norm | BatchNorm1d         | 60    
2 | backbone               | FeedForwardBackbone | 19.0 M
3 | output_layer           | Linear              | 1.0 K 
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
19.0 M    Trainable params
0         Non-trainable params
19.0 M    Total params
76.028    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [13]:
tabular_model.save_model('./saved_models/tab_model')

In [14]:
loaded_model = TabularModel.load_from_checkpoint('./saved_models/tab_model/')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [15]:
# iris-flower dataset

In [20]:
# reading data
data2=pd.read_csv('Iris.csv')
data2.drop(['Id'],axis=1,inplace=True)
data2['Species'] = data2['Species'].astype('category').cat.codes# encoding labels

In [21]:
train, test, target_col, cat_col_names, num_col_names  = load_classification_data(data2,'Species',0.2)

In [22]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=16,
    max_epochs=100,
    auto_select_gpus = True,
    gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="4096-4096-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3,
    metrics=["accuracy"]
)
tabular_model2 = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

In [23]:
tabular_model2.fit(train=train, test=test)

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 0     
1 | normalizing_batch_norm | BatchNorm1d         | 8     
2 | backbone               | FeedForwardBackbone | 18.9 M
3 | output_layer           | Linear              | 1.5 K 
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
75.604    Total estimated model params size (MB)
Global seed set to 42


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
-------------------------

LR finder stopped early after 84 steps due to diverging loss.


--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [25]:
#pima-indian-diabetes dataset

In [26]:
# reading data
data3=pd.read_csv('diabetes.csv')
# data3.drop('id',axis=1,inplace=True)
data3['quality'] = data3['Outcome'].astype('category').cat.codes# encoding labels

In [27]:
train, test, target_col, cat_col_names, num_col_names= load_classification_data(data3,'quality',0.2)

In [28]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=32,
    max_epochs=100,
    gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="1024-1024-32",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3,
    metrics=["accuracy"]
)
tabular_model3 = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

In [29]:
tabular_model3.fit(train=train, test=test)

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 6     
1 | normalizing_batch_norm | BatchNorm1d         | 16    
2 | backbone               | FeedForwardBackbone | 1.1 M 
3 | output_layer           | Linear              | 66    
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.375     Total estimated model params size (MB)
Global seed set to 42


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
-------------------------

LR finder stopped early after 98 steps due to diverging loss.
Restored states from the checkpoint file at /content/lr_find_temp_model.ckpt
Learning rate set to 1.5848931924611133e-07
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 6     
1 | normalizing_batch_norm | BatchNorm1d         | 16    
2 | backbone               | FeedForwardBackbone | 1.1 M 
3 | output_layer           | Linear              | 66    
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.375     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

...

Training: 1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [30]:
tabular_model3.evaluate(test)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.7189542651176453}
--------------------------------------------------------------------------------


[{'test_accuracy': 0.7189542651176453}]

In [31]:
#heart-disease-uci dataset

In [32]:
# reading data
data4=pd.read_csv('heart.csv')
# data3.drop('id',axis=1,inplace=True)
data4['target'] = data4['target'].astype('category').cat.codes# encoding labels

In [33]:
train, test, target_col, cat_col_names, num_col_names= load_classification_data(data4,'target',0.2)

In [34]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=32,
    max_epochs=100,
    gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="4096-4096-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3,
    metrics=["accuracy"]
)
tabular_model4 = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

In [35]:
tabular_model4.fit(train=train, test=test)

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 75    
1 | normalizing_batch_norm | BatchNorm1d         | 10    
2 | backbone               | FeedForwardBackbone | 19.0 M
3 | output_layer           | Linear              | 1.0 K 
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
19.0 M    Trainable params
0         Non-trainable params
19.0 M    Total params
75.914    Total estimated model params size (MB)
Global seed set to 42


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
-------------------------

LR finder stopped early after 88 steps due to diverging loss.
Restored states from the checkpoint file at /content/lr_find_temp_model.ckpt
Learning rate set to 1.3182567385564074e-07
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 75    
1 | normalizing_batch_norm | BatchNorm1d         | 10    
2 | backbone               | FeedForwardBackbone | 19.0 M
3 | output_layer           | Linear              | 1.0 K 
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
19.0 M    Trainable params
0         Non-trainable params
19.0 M    Total params
75.914    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [36]:
tabular_model4.evaluate(test)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.6833333373069763}
--------------------------------------------------------------------------------


[{'test_accuracy': 0.6833333373069763}]

In [37]:
#winequality-red dataset

In [38]:
# reading data
data5=pd.read_csv('winequality-red.csv')
data5['quality'] = data5['quality'].astype('category').cat.codes# encoding labels

In [39]:
train, test, target_col, cat_col_names, num_col_names= load_classification_data(data5,'quality',0.2)

In [40]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=32,
    max_epochs=100,
    gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="4096-4096-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3,
    metrics=["accuracy"]
)
tabular_model5 = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

In [41]:
tabular_model5.fit(train=train, test=test)

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 0     
1 | normalizing_batch_norm | BatchNorm1d         | 22    
2 | backbone               | FeedForwardBackbone | 18.9 M
3 | output_layer           | Linear              | 3.1 K 
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
75.725    Total estimated model params size (MB)
Global seed set to 42


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------


LR finder stopped early after 96 steps due to diverging loss.
Restored states from the checkpoint file at /content/lr_find_temp_model.ckpt
Learning rate set to 2.2908676527677725e-05
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 0     
1 | normalizing_batch_norm | BatchNorm1d         | 22    
2 | backbone               | FeedForwardBackbone | 18.9 M
3 | output_layer           | Linear              | 3.1 K 
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
75.725    Total estimated model params size (MB)


--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [42]:
tabular_model5.evaluate(test)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.5297805666923523}
--------------------------------------------------------------------------------


[{'test_accuracy': 0.5297805666923523}]